## Introduction

Here we are trying to make a binary classification between those whom MoCA score worthens or it does not based on just their year 0 and 1 DatScan
🥑

# **Setup**

1. **Import libraries**

2. **Connect to google drive**

3. **Initiate GPU**

In [ ]:
!pip install classification-models-3D
!pip install keras_applications
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from google.colab import drive
import nibabel as nib
from scipy import ndimage
import random
from PIL import Image
from matplotlib import cm
import pandas as pd
import glob as glob
from os.path import exists
from IPython.display import clear_output 
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
from classification_models_3D.tfkeras import Classifiers

#Connect Google drive
drive.mount('/content/drive')

#Initialize GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else: print('Found GPU at: {}'.format(device_name))

In [ ]:
import os
files = glob.glob("/content/drive/MyDrive/datscans/*.nii")
for filee in files:
  img = nib.load(filee)
  if str(img.shape)=="(91, 109, 91, 1)":
   print(filee.split("/")[-1]+" "+str(img.shape))
  else:
    print(filee.split("/")[-1]+" "+"NOT CORRECT SHAPE")
    os.remove(filee)


In [ ]:
def read_dicom_file(filepath):
    dicom = pydicom.dcmread(filepath)
    return dicom

In [ ]:
def read_file(filepath):
    """Read and load volume"""
    scan = read_dicom_file(filepath)
    scan = scan.pixel_array
    return scan

def crop (image,x1,x2,y1,y2,z1,z2):
    """Crop image"""
    if (x2-x1)%2 != 0: x1 = x1 - 1
    if (y2-y1)%2 != 0: y1 = y1 - 1
    if (z2-z1)%2 != 0: z1 = z1 - 1

    xdiff = round((22 - x2 + x1)/2)
    ydiff = round((28 - y2 + y1)/2)
    zdiff = round((40 - z2 + z1)/2)

    #volume = image[y1 - ydiff : y2 + ydiff,x1 - xdiff : x2 + xdiff, z1 - zdiff : z2 + zdiff]
    volume = image[:,:, z1 - zdiff : z2 + zdiff]
    return volume

def normalise_zero_one(image):  
    """Image normalisation. Normalises image to fit [0, 1] range."""
    image = image.astype(np.float32)
    minimum = np.min(image)
    maximum = np.max(image)
    if maximum > minimum:
        ret = (image - minimum) / (maximum - minimum)
    else:
        ret = image * 0.
    return ret


def process_scan(dim_path):
    """Read and resize volume"""
    volume = read_file(dim_path.get("path"))
    volume = crop(volume,dim_path.get("x1"),dim_path.get("x2"),dim_path.get("y1"),dim_path.get("y2"),dim_path.get("z1"),dim_path.get("z2"))
    volume = normalise_zero_one(volume)
    return volume


In [ ]:
paths = glob.glob('/content/drive/MyDrive/P202204050001/Codes and datasets/Datasets/preprocessed_dataset/year0/Segmented_SC/DICOM_SegmentedImages/*/image_Whole3D.dcm')
len(paths)

In [ ]:
!pip install pydicom
import pydicom

In [ ]:
counter =  0
xdiffs = []
ydiffs = []
zdiffs = []
x1s =[]
y1s =[]
z1s =[]
x2s =[]
y2s =[]
z2s =[]
my_dicts = []
while counter < len(paths):
  print(round(100*counter / len(paths)),"% proccesing")
  dicom = read_dicom_file(paths[counter])
  arr = dicom.pixel_array
  clear_output()

  a = 0
  while a < arr.shape[0]:
    if 255 in arr[a,:,:]:
      x1 = a
      x1s.append(x1)
      break;
    a+=1

  la = arr.shape[0]
  while la > 0:
    if 255 in arr[la-1,:,:]:
      x2 = la
      x2s.append(x2)
      break;
    la-=1

  xdiff = x2-x1
  xdiffs.append(abs(xdiff))

  b = 0
  while b < arr.shape[1]:
    if 255 in arr[:,b,:]:
      y1 = b
      y1s.append(y1)
      break;
    b+=1

  lb = arr.shape[1]
  while lb > 0:
    if 255 in arr[:,lb-1,:]:
      y2 = lb
      y2s.append(y2)
      break;
    lb-=1

  ydiff = y2-y1
  ydiffs.append(abs(ydiff))

  c = 0
  while c < arr.shape[2]:
    if 255 in arr[:,:,c]:
      z1 = c
      z1s.append(z1)
      break;
    c+=1

  lc = arr.shape[2]
  while lc > 0:
    if 255 in arr[:,:,lc-1]:
      z2 = lc
      z2s.append(z2)
      break;
    lc-=1

  zdiff = z2-z1
  zdiffs.append(abs(zdiff))
  my_dict = {}
  my_dict['z1'] = z1
  my_dict['x1'] = x1
  my_dict['y1'] = y1
  my_dict['z2'] = z2
  my_dict['x2'] = x2
  my_dict['y2'] = y2
  my_dict['path'] = paths[counter]
  my_dicts.append(my_dict)
  counter += 1

print(len(my_dicts))
print(np.amin(z1s)," : z1")
print(np.amin(y1s)," : y1")
print(np.amin(x1s)," : x1")

print(np.amax(z2s)," : z2")
print(np.amax(x2s)," : x2")
print(np.amax(y2s)," : y2")

print(np.amax(xdiffs)," : x")
print(np.amax(ydiffs)," : y")
print(np.amax(zdiffs)," : z")

# **Loading data and preprocessing**

Preprocessing algorithm is inherited from diffrent papers


* **Input size** :

  Diffrent ROI sizes have been used in previous studies.

  1.   79 x 95 x 69 -> [Paper](https://linkinghub.elsevier.com/retrieve/pii/S0895611120301051)
  2.   91 x 109 x 91 -> Most papers
  3.   32 X 48 X 16 -> [Paper](https://jnm.snmjournals.org/content/59/supplement_1/29)
  
  But in this study entire images are taken into consideration, and the network is trained accordingly. This, in turn, allows the network to be more robust and efficient in classifying PD patients from healthy controls patients because classification is performed based on the whole SPECT image of the brain. as suggested in [this](https://pubmed.ncbi.nlm.nih.gov/33279760/) study.◎ *and we have further remove 16 slices from the bottom of axial slices that mostly were damaged and not rendered correctly after reconstruction. then we removed a total of 18 slices from both sagital axis ends that contain no data and it helps the network to manage ram better.*◎


* **Normalization** :

  Another preproccecing method that we are applying here is intensity normalization which is used in [this](https://arxiv.org/abs/2008.00238) paper

In [ ]:

# def read_nifti_file(filepath):
#     """Read and load volume"""
#     # Read file
#     scan = nib.load(filepath)
#     # Get raw data
#     scan = scan.get_fdata()
#     return scan
 
# def normalise_zero_one(image):
  
#     """Image normalisation. Normalises image to fit [0, 1] range."""
#     image = image.astype(np.float32)
#     minimum = np.min(image)
#     maximum = np.max(image)
#     if maximum > minimum:
#         ret = (image - minimum) / (maximum - minimum)
#     else:
#         ret = image * 0.
#     return ret


# def process_scan(path):
#     """Read and resize volume"""
#     # Read scan
#     volume = read_nifti_file(path)
#     # Resize width, height and depth
#     # volume = volume[0:91,9:100,16:91]
#     # Normalize
#     volume = normalise_zero_one(volume)
  
#     return volume


# **Importing Nifti files from tables**
Let's read the paths of the DAT scans from the directories.
And do the main preprocessing on both worsened and not not worthened cognitive impairment patients.
We also read feature from the coresponding table.
predictor features are chosen from [this](https://link.springer.com/article/10.1007/s00415-020-09757-9) study.
It includes:
1. Age at onset
2. APOE
3. RBD
4. Hallucination
5. PIGD

in order to cover all possible cognitive impairment scales we use three diffrent database in all which we calculated year n and n+5 cognitive scores. those three methods are listed below and are inherited from [this](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0175674) paper.

1. MoCA cut-off for PD of <26  ---> moca
2. Using the detailed cognitive battery, cognitive impairment was defined as at least two test scores >1.5 standard deviations below the standardized mean score, a level of impairment within the recommend range (>1.0–2.0) of standard deviations below the mean to support a PD-MCI diagnosis[19]. Single scores were generated for each test, except for the HVLT-T, for which two scores were used ---> neuro
3. The site investigator’s clinical diagnosis of cognitive impairment (PD-MCI or PDD) versus no cognitive impairment was made annually. Each site investigator was provided an instruction sheet that outlined how to assess cognitive decline, functional impairment, and general interpretation of cognitive tests to make a diagnosis of PD-MCI or PDD ---> cog

In [ ]:
# labels_list = []
# import pydicom._storage_sopclass_uids
# from pydicom import dcmread

# scans_list = []
# excel_path = "/content/drive/MyDrive/P202204050001/Codes and datasets/Datasets/arman_dat_zero_moca_four.xlsx"
# moca_table = pd.read_excel(excel_path)
# for dim_path in my_dicts:
#     if 'Segmented_SC_orginal' not in dim_path["path"]:
#       newpath = dim_path["path"].replace('Segmented_SC','Segmented_SC_orginal').replace('image_Whole3D.dcm','data3D.dcm')  
#       dim_path.update({"path": newpath})
#       print(dim_path["path"])

# for i, row in moca_table.iterrows():
#     passs = []
#     for my_path in my_dicts:
#       if str(int(row.patno)) == my_path['path'].split('/')[-2]:
#         print("image : "+ str(row.patno)+" At "+str(round(i/len(moca_table)*100))+ " % of preprocessing.")
#         procesed = process_scan(dim_path)
#         labels_list.append(int(row.moca_class))
#         scans_list.append(procesed)

# scans = np.asarray(scans_list)
# labels = np.asarray(labels_list)

# print("Scans: " + str(len(scans)))
# print("labels: " + str(len(labels)))

# print("Images dimentions are: " +str(scans[0].shape))


In [ ]:
# for i in range(scans.shape[1]):
#   print("Original image : ")
#   plt.imshow(scans[10][i, :, :], cmap="gist_gray")
#   plt.show()

In [ ]:
feature = ""
cognitive_scale = "cog"

worsened_scans_list = []
worsened_labels_list = []

worsened_feature_list =[]
not_worsened_feature_list =[]

not_worsened_paths = []
worsened_paths = [] 

not_worsened_scans_list = []
not_worsened_labels_list = []

cog_dat_table = pd.read_csv("/content/drive/MyDrive/progress_visualizer-master/"+cognitive_scale+
                            "_dat_feature/main_tables/"+cognitive_scale+"_dat_"+feature+"main.csv")
for dim_path in my_dicts:
    if 'Segmented_SC_orginal' not in dim_path["path"]:
      newpath = dim_path["path"].replace('Segmented_SC','Segmented_SC_orginal').replace('image_Whole3D.dcm','data3D.dcm')  
      dim_path.update({"path": newpath})
      print(dim_path["path"])
      
for i, row in cog_dat_table.iterrows():
  passs = []
  for my_path in my_dicts:
    if str(int(row.PATNO)) == my_path['path'].split('/')[-2]:
      path = "/content/drive/MyDrive/year_zero_and_one_datscans/I"+str(row.IMGID)+".nii"
      #clear_output()
      #print("image : "+ str(row.IMGID)+" At "+str(round(i/len(cog_dat_table)*100))+ " % of preprocessing.")
  
      if row.CONUMDIFF < 0:
        #print("-1")
        if exists(path):
          procesed = process_scan(my_path)
          print(procesed.shape)
          worsened_scans_list.append(procesed)
          worsened_paths.append(path)
          if feature != "":
            worsened_feature_list.append(row.FEATURE)
          worsened_labels_list.append(0)
      else:
        #print("+1")
        if exists(path):
          procesed = process_scan(my_path)
          print(procesed.shape)
          not_worsened_scans_list.append(procesed)
          not_worsened_paths.append(path)
          if feature != "":
            not_worsened_feature_list.append(row.FEATURE)
          not_worsened_labels_list.append(1)

if feature != "":
  worsened_feature = np.asarray(worsened_feature_list)

worsened_scans = np.asarray(worsened_scans_list)
worsened_labels = np.asarray(worsened_labels_list)

if feature != "":
  not_worsened_feature = np.asarray(not_worsened_feature_list)

not_worsened_scans = np.asarray(not_worsened_scans_list)
not_worsened_labels = np.asarray(not_worsened_labels_list) 


print("Worsened Scans: " + str(len(worsened_scans)))
print("Not Worsened Scans: " + str(len(not_worsened_scans)))
print("Images dimentions are: " +str(not_worsened_scans[0].shape))

In [ ]:
for i in range(worsened_scans.shape[1]):
  print("Original image : ")
  plt.imshow(worsened_scans[10][i, :, :], cmap="gist_gray")
  plt.show()

# **Build train and validation datasets**
for model we need two input which are image and feature and and an input which is label in here. we also devide data to pre-augmneted train part which contains 2/3 of whole dataset and validation part which is 1/3 of the whole dataset.

In [ ]:
image_train = np.concatenate((not_worsened_scans[:round(len(not_worsened_scans)-(len(not_worsened_scans)/2))],
                          worsened_scans[:round(len(worsened_scans)-(len(worsened_scans)/2))]), axis=0)
if feature != "":
  feature_train = np.concatenate((not_worsened_feature[:round(len(not_worsened_feature)-(len(not_worsened_feature)/2))],
                          worsened_feature[:round(len(worsened_feature)-(len(worsened_feature)/2))]), axis=0)

label_train = np.concatenate((not_worsened_labels[:round(len(not_worsened_labels)-(len(not_worsened_labels)/2))],
                          worsened_labels[:round(len(worsened_labels)-(len(worsened_labels)/2))]), axis=0)

image_val = np.concatenate((not_worsened_scans[round(len(not_worsened_scans)-(len(not_worsened_scans)/2)):],
                          worsened_scans[round(len(worsened_scans)-(len(worsened_scans)/2)):]), axis=0)
if feature != "":
  feature_val = np.concatenate((not_worsened_feature[round(len(not_worsened_feature)-(len(not_worsened_feature)/2)):],
                          worsened_feature[round(len(worsened_feature)-(len(worsened_feature)/2)):]), axis=0)

label_val = np.concatenate((not_worsened_labels[round(len(not_worsened_labels)-(len(not_worsened_labels)/2)):],
                          worsened_labels[round(len(worsened_labels)-(len(worsened_labels)/2)):]), axis=0)

if feature != "":
  print("Train size : "+str(image_train.shape[0])+" "+ str(feature_train.shape[0])+" "+str(label_train.shape[0]))
  print("Validation size : "+str(image_val.shape[0])+" "+ str(feature_val.shape[0])+" "+str(label_val.shape[0]))
else:
  print("Train size : "+str(image_train.shape[0])+" "+str(label_train.shape[0]))
  print("Validation size : "+str(image_val.shape[0])+" "+str(label_val.shape[0]))

# **Data Augmentation**
we augmented iamges just with flipping images left to write as suggested in most papers such as [this](https://pubmed.ncbi.nlm.nih.gov/33279760/) one. in order to maintain dataset size and labels we then double the size of labels and features with no further augmentation.

In [ ]:
lrflip_augmented_images_list = []
udflip_augmented_images_list = []

for image in image_train:
  lrflip_augmented_image = np.fliplr(image)
  lrflip_augmented_images_list.append(lrflip_augmented_image)
  udflip_augmented_image = np.flipud(image)
  udflip_augmented_images_list.append(udflip_augmented_image)


print("Original image : ")
plt.imshow(np.squeeze(image_train[2][10, :, :]), cmap="magma")
plt.show()

print("Augmented images : ")
udflip_augmented_images = np.asarray(udflip_augmented_images_list)
lrflip_augmented_images = np.asarray(lrflip_augmented_images_list)

plt.imshow(np.squeeze(lrflip_augmented_images[2][10, :, :]), cmap="magma")
plt.show()

plt.imshow(np.squeeze(udflip_augmented_images[2][10, :, :]), cmap="magma")
plt.show()

if feature != "":
  feature_train = np.concatenate((feature_train,feature_train,feature_train), axis=0)

label_train = np.concatenate((label_train,label_train), axis=0)
image_train = np.concatenate((image_train,lrflip_augmented_images), axis=0)


if feature != "":
  print("Train size : "+str(image_train.shape[0])+" "+ str(feature_train.shape[0])+" "+str(label_train.shape[0]))
  print("Validation size : "+str(image_val.shape[0])+" "+ str(feature_val.shape[0])+" "+str(label_val.shape[0]))
else:
  print("Train size : "+str(image_train.shape[0])+" "+str(label_train.shape[0]))
  print("Validation size : "+str(image_val.shape[0])+" "+str(label_val.shape[0]))

#**Our own 3D image classification model**

Inspired from [this](https://10.1007/978-3-030-59354-4_15) paper, we design a 17 layer 3D CNN which comprises four 3D con-volutional (CONV) layers with two layers consisting of 64 filters followed by 128and 256 filters all with a kernel size of 3 × 3 × 3. Each CONV layer is followed bya max-pooling (MAXPOOL) layer with a stride of 2 and ReLU activation whichends with batch normalization (BN) layer. Essentially, our feature extractionblock consists of four CONV-MAXPOOL-BN modules. The final output fromthe feature extraction block is flattened and passed to a fully connected layerwith 512 neurons. We use an effective dropout rate of 60% similar to [31]. Due toa coding error, we implement this using two dropout layers [34]. The output isthen carried to a dense layer of 2 neurons with softmax activation for the binaryclassification problem. The network architecture is shown in Fig. 2.We consider keeping the network relatively simple to avoid overparameter-ization [32] problems with only 10,658,498 learnable parameters. This is alsomotivated by the fewer number of training samples and the memory challengesassociated with it. if you have limited data, then it’s likely that you will also have to limit the complexity of the ML models you use, since models with many parameters, like deep neural networks, can easily overfit small data sets. Either way, it’s important to identify this issue early on, and come up with a suitable (and defensible) strategy to mitigate it.

In [ ]:
def get_own_model(width, height, depth):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=16, kernel_size=3, activation="relu",padding="same")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=32, kernel_size=3, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    own_output = layers.Dense(units=512, activation="relu")(x)
   
    # Define the model.
    model = keras.Model(inputs, own_output, name="3dcnn")
    return model

In [ ]:
def get_own_standalone_model(width, height, depth):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=16, kernel_size=3, activation="relu",padding="same")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=32, kernel_size=3, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(units=1, activation="sigmoid")(x)
   
    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

In [ ]:
def get_rahmim_model(width, height, depth):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=16, kernel_size=7,strides=4, activation="relu",padding="same")(inputs)
    x = layers.MaxPool3D(pool_size=3,strides=2)(x)
    x = layers.Conv3D(filters=32, kernel_size=5,strides=1, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=3,strides=2)(x)
    x = layers.Conv3D(filters=64, kernel_size=3,strides=1, activation="relu",padding="same")(x)
    x = layers.MaxPool3D(pool_size=3,strides=2)(x)
    x = layers.GlobalAveragePooling3D()(x)
    rahmim_output = layers.Dense(units=512, activation="relu")(x)
   
    # Define the model.
    model = keras.Model(inputs, rahmim_output, name="3dcnn")
    return model

#**3D Image branch part**
We used two diffrent CNN architecture without their pretrained weight for our model as they were previously shown to have a good performance in 2D image classification.
1. VGG16
2. ResNet50

In [ ]:
from classification_models_3D.models.resnext import ResNeXt50
from classification_models_3D.models.vgg16 import VGG16
from classification_models_3D.models.inception_v3 import InceptionV3

model_name = 'rahmim'

if model_name == 'resnet50':
  ResNet50, preprocess_input = Classifiers.get('resnet50')
  image_model = ResNet50(input_shape=(22, 28, 40, 1),weights=None)
  
elif model_name == 'inceptionv3':
  InceptionV3, preprocess_input = Classifiers.get('inceptionv3')
  image_model = InceptionV3(input_shape=(91, 91, 75, 1),weights=None)

elif model_name == 'own':
  image_model = get_own_model(width=91, height=109, depth=91)

elif model_name == 'rahmim':
  image_model = get_rahmim_model(width=22, height=28, depth=40)

elif model_name == "own_std":
  image_model = get_own_standalone_model(width=91, height=109, depth=91)

#**Feature branch model part ⭕️**
here we make a simple 4 layer model to train on feature numerical data.

In [ ]:
feature_model = keras.models.Sequential()
feature_model.add(layers.Dense(16, activation='relu', input_shape=(1,)))
feature_model.add(layers.Dense(64, activation='relu'))
feature_model.add(layers.Dense(256, activation='relu'))
feature_model.add(layers.Dense(2048, activation='relu'))

#**Concatenation ⭕️**
Concatenate both models and add some final layers at the end. and plot the final model scheme

In [ ]:
combinedInput = layers.concatenate([image_model.output, feature_model.output])

x = layers.Dense(units=512, activation="relu")(combinedInput)
x = layers.Dense(units=256, activation="relu")(x)
x = layers.Dense(units=128, activation="relu")(x)
x = layers.Dense(units=64, activation="relu")(x)
x = layers.Dense(units=16, activation="relu")(x)
x = layers.Dense(units=4, activation="relu")(x)
x = layers.Dropout(0.3)(x)
combined_output = layers.Dense(units=1, activation="sigmoid")(x)

if feature != "":
  final_model = keras.Model(inputs=[image_model.input, feature_model.input], outputs=combined_output.output)
else: 
  final_model = keras.Model(inputs=image_model.input, outputs=image_model.output)

keras.utils.plot_model(final_model, show_shapes=True)

#**Running model ⭕️**
Compiling and running model
with Adam Optimizer and learning rate of 0.0001

In [ ]:
N_EPOCHS = 4
!pip install tensorflow-addons
import datetime
import tensorflow_addons as tfa

es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto', patience=40)
mc = keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)
tb = keras.callbacks.TensorBoard(log_dir='./logs_v2', write_images=True, write_graph=True)
logdir = os.path.join("logs2", datetime.datetime.now().strftime("%Y/%m/%d-%H:%M:%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

final_model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy','percision' 'AUC'])
if feature != "":
  history = final_model.fit([image_train, feature_train],label_train,batch_size=4,
                 epochs=N_EPOCHS, validation_data=([image_val, feature_val],label_val), callbacks=[tensorboard_callback, es, mc])
else:
  history = final_model.fit(image_train,label_train,batch_size=4,
                 epochs=N_EPOCHS, validation_data=(image_val,label_val), callbacks=[tensorboard_callback, es, mc])
print(history)
model_json = final_model.to_json()
with open("/content/drive/MyDrive/progress_visualizer-master/model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs2